<h2>For each Student and test, identify if their marks increased or decreased from the previous tests.
</h2>

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr,col

spark=SparkSession.builder.appName('Student-Marks').getOrCreate()

In [8]:
from pyspark.sql.functions import to_date,sum

data = [(2,'Max Ruin','Subject1',63,1,'2022-01-02'),
(3,'Arnold','Subject1',95,1,'2022-01-02'),
(4,'Krish Star','Subject1',61,1,'2022-01-02'),
(5,'John Mike','Subject1',91,1,'2022-01-02'),
(4,'Krish Star','Subject2',71,1,'2022-01-02'),
(3,'Arnold','Subject2',32,1,'2022-01-02'),
(5,'John Mike','Subject2',61,2,'2022-11-02'),
(1,'John Deo','Subject2',60,1,'2022-01-02'),
(2,'Max Ruin','Subject2',84,1,'2022-01-02'),
(2,'Max Ruin','Subject3',29,3,'2022-01-03'),
(5,'John Mike','Subject3',98,2,'2022-11-02')] 
schema = """studentid int,
 studentname string,
 subject string,
 marks int,
 testid int,
 testdate string"""
df=spark.createDataFrame(data,schema)
df=df.withColumn("testdate",to_date(df.testdate))
df.printSchema()

root
 |-- studentid: integer (nullable = true)
 |-- studentname: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- testid: integer (nullable = true)
 |-- testdate: date (nullable = true)



In [12]:
from pyspark.sql.window import Window
window_spec=Window.partitionBy(df.studentid).orderBy(df.testid.asc(),df.testdate.asc(),df.subject.asc())

In [16]:
from pyspark.sql.functions import lag
remarks_df=df.withColumn('prev_exam_marks',lag(df.marks).over(window_spec))\
.withColumn("remarks",expr("case when marks>=prev_exam_marks then 'INCREASED' else 'DECREASED'end ")).show()

+---------+-----------+--------+-----+------+----------+---------------+---------+
|studentid|studentname| subject|marks|testid|  testdate|prev_exam_marks|  remarks|
+---------+-----------+--------+-----+------+----------+---------------+---------+
|        1|   John Deo|Subject2|   60|     1|2022-01-02|           NULL|DECREASED|
|        2|   Max Ruin|Subject1|   63|     1|2022-01-02|           NULL|DECREASED|
|        2|   Max Ruin|Subject2|   84|     1|2022-01-02|             63|INCREASED|
|        2|   Max Ruin|Subject3|   29|     3|2022-01-03|             84|DECREASED|
|        3|     Arnold|Subject1|   95|     1|2022-01-02|           NULL|DECREASED|
|        3|     Arnold|Subject2|   32|     1|2022-01-02|             95|DECREASED|
|        4| Krish Star|Subject1|   61|     1|2022-01-02|           NULL|DECREASED|
|        4| Krish Star|Subject2|   71|     1|2022-01-02|             61|INCREASED|
|        5|  John Mike|Subject1|   91|     1|2022-01-02|           NULL|DECREASED|
|   